In [ ]:
import oat_python as oat

import plotly.graph_objects as go
import numpy as np
import sklearn
import sklearn.metrics

# Homology of a Dowker complex

In this example we will
- generate a point cloud
- generate a collection of subsets (some would call this a *cover*, others would call it a *hypergraph*)
- compute the homology of the associated Dowker complex
- analyze cycle representatives
- plot cycle representatives

## Generate a point cloud

In [ ]:
#   SET PARAMETER VALUES
npoints             =   200
maxdis              =   None
maxdim              =   1

#   GENERATE CLOUD FROM RANDOM SEED
cloud              =   oat.point_cloud.slice_of_sphere( npoints=npoints, randomize=True, random_seed=0, )
cloud              =   cloud + 0.2 * ( np.random.rand( *cloud.shape ) - 0.5 )
cloud              =   cloud[ cloud[:,0].argsort() ] # for convenience, sort rows by x-coordinate

#   PLOT

#   the point cloud
data                =   [ go.Scatter3d( name="Cloud",
                                        mode="markers",                                        
                                        x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], 
                                        marker=dict(size=5, color=-cloud[:,2], colorscale="Peach") # Aggrnyl, Oryel, Deep, Reds
                                        ) ]

#   a wire sphere, for visibility
for n, trace in enumerate(oat.plot.wire_sphere3d(0,0,0,1, nlattitude=10, nlongitude=20)):
    trace.update( name="Wire", legendgroup="1", showlegend = n==0, line=dict(color="white", width=3), opacity=0.3 )
    data.append(trace)

fig = go.Figure(data=data)
fig.update_layout( title=dict(text="Sphere with noise - try rotating!"), width=800, height=800, )

fig.update_layout(scene = dict(     
    bgcolor="black",
    xaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    yaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    zaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
))

fig.show()

## Choose a cover

In [ ]:
from sklearn.neighbors import NearestNeighbors


radius_net          =   0.2; # we'll make epsilon net with this value of epsilon
radius_neighbor     =   0.5; # hyperedges will be neighborhoods of vertices, with this radius

#   COMPUTE AN EPSILON NET

net, curve          =   oat.dissimilarity.farthest_point_with_cloud( 
                            cloud   =   cloud, 
                            epsilon =   radius_net,
                        )

#   COMPUTE THE CORRESPONDING COVER

net_points          =   NearestNeighbors(n_neighbors=1, algorithm='ball_tree').fit( cloud[net] ) # data structure holding the whole cloud
cover               =   [ sorted(list(x)) for x in net_points.radius_neighbors( cloud, radius=radius_neighbor, return_distance=False, ) ]

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

#   CLOUD
data                =   [ go.Scatter3d( name="Cloud",
                                        mode="markers",                                        
                                        x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], 
                                        marker=dict(size=5, color=-cloud[:,2], colorscale="Peach") # Aggrnyl, Oryel, Deep, Reds
                                        ) ]
#   WIRE
for n, trace in enumerate(oat.plot.wire_sphere3d(0,0,0,1, nlattitude=10, nlongitude=20)):
    trace.update( name="Wire", legendgroup="1", showlegend = n==0, line=dict(color="white", width=3), opacity=1.0 )
    data.append(trace)

#   HYPEREDGES
for counter, n in enumerate(net):
    trace, x, y, z = oat.plot.surface_sphere( x=cloud[n][0], y=cloud[n][1], z=cloud[n][2], radius=radius_neighbor, resolution=20 )
    trace.update(opacity=1.0, showscale=False, showlegend=True, name=f"Edge {counter}" )
    if counter % 10 != 0: trace.update(visible='legendonly',) 
    data.append(trace)

fig = go.Figure(data)
fig.update_layout(width=800, height=800 )
fig.update_layout(
    title = f"Toggle the hyperedges!",
    scene = dict(
        aspectratio=go.layout.scene.Aspectratio(x=2, y=2, z=2),
        xaxis = dict(range=[-2.5,2.5],),
        yaxis = dict(range=[-2.5,2.5],),
        zaxis = dict(range=[-2.5,2.5],),                
    )
)
fig.update_layout(scene = dict(     
    bgcolor="black",
    xaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    yaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    zaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
))
fig.show()

## Compute homology

We'll compute the homology of
- the dual hypergraph; that is, the hypergraph where vertices are balls, and for each vertex `v` we have a hyperedge that contains every ball to which `v` belongs
- equivalently, the witness complex where every point is a witness and net points are landmarks

The homology solver only accepts hypergraphs represented by a list of lists of integers, currently.  If your hypergraph has a different format (e.g., if vertices are strings), then you can use some built-in tools to help translate back and forth between this format and list-of-list format; see the [rbs_reduced](rbs_reduced.ipynb) notebook for examples.

In [ ]:
#   FACTOR THE BOUNDARY MATRIX

factored    =   oat.rust.FactoredBoundaryMatrixDowker( 
                    dowker_simplices            =   cover, 
                    max_homology_dimension      =   2
                )

#   BETTI STATISTICS

print("\n\nBetti statistics\n")
display( factored.betti() )

#   HOMOLOGY

print("\nHomology\n")
homology    =   factored.homology()
display( homology )

#   CYCLE REPRESENTATIVE

print("\nCycle representative\n")
display( homology["cycle representative"][1] )


## Plot a 2-cycle

In [ ]:
#   CLOUD
data                =   [   go.Scatter3d( name="Cloud",
                                        mode="markers",                                        
                                        x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], 
                                        marker=dict(
                                            color=-cloud[:,2], 
                                            colorscale="Peach"
                                        ),
                                        # visible="legendonly" 
                            ) 
                        ]

#   CYCLE
data_cycle = []
coo = cloud[net] # coo stands for "coordinate oracle"
triangles = homology["cycle representative"][1]["simplex"].tolist() # list the triangles in the cycle
for count, triangle in enumerate(triangles):
    trace = oat.plot.triangle__trace3d( triangle, coo=coo ) # make the trace for each simplex
    trace.update(intensity=cloud[ np.array([net[p] for p in triangle]),[2]], showscale=False, cmin=-1, cmax=1, colorscale="Aggrnyl")
    trace.update(name=f"simplex {triangle}", text=f"simplex {triangle}", opacity=1.0, showlegend=True) # customize
    data.append(trace) # append to the data group


#   TRIANGLE BOUNDARIES
edges = list(oat.simplex.dnfaces( triangles, facedim=1))
for edge in edges:
    trace   =   oat.plot.edge__trace3d( edge, coo=coo )
    trace.update( showlegend=False, hoverinfo='none', line=dict(width=5, color="white"))
    data.append(trace)

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Toggle the legend entries to hide/reveal individual simplices!", height=1000,width=1000, ) 
fig.update_layout(scene = dict(     
    bgcolor="black",
    xaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    yaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    zaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
))
fig.show()

# Plot another 2-cycle

In [ ]:
#   CLOUD
data                =   [   go.Scatter3d( name="Cloud",
                                        mode="markers",                                        
                                        x=cloud[:,0],y=cloud[:,1],z=cloud[:,2], 
                                        marker=dict(
                                            color=-cloud[:,2], 
                                            colorscale="Peach"
                                        ),
                                        # visible="legendonly" 
                            ) 
                        ]

#   CYCLE
data_cycle = []
coo = cloud[net] # coo stands for "coordinate oracle"
triangles = homology["cycle representative"][2]["simplex"].tolist() # list the triangles in the cycle
for count, triangle in enumerate(triangles):
    trace = oat.plot.triangle__trace3d( triangle, coo=coo ) # make the trace for each simplex
    trace.update(intensity=cloud[ np.array([net[p] for p in triangle]),[2]], showscale=False, cmin=-1, cmax=1, colorscale="Aggrnyl")
    trace.update(name=f"simplex {triangle}", text=f"simplex {triangle}", opacity=1.0, showlegend=True) # customize
    data.append(trace) # append to the data group


#   TRIANGLE BOUNDARIES
edges = list(oat.simplex.dnfaces( triangles, facedim=1))
for edge in edges:
    trace   =   oat.plot.edge__trace3d( edge, coo=coo )
    trace.update( showlegend=False, hoverinfo='none', line=dict(width=5, color="white"))
    data.append(trace)

#   PLOT

fig = go.Figure( data )
fig.update_layout(title="Toggle the legend entries to hide/reveal individual simplices!", height=1000,width=1000, ) 
fig.update_layout(scene = dict(     
    bgcolor="black",
    xaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    yaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
    zaxis = dict(showgrid = False,showticklabels = False, showline=False, zeroline=False, backgroundcolor="black"),
))
fig.show()